In [1]:
from google.colab import files
uploaded=files.upload()

Saving ar.dic to ar.dic
Saving ar.aff to ar.aff


In [ ]:
file_name="ar.aff"
uploaded[file_name].decode("utf-8")
file_name2="ar.dic"
uploaded[file_name2].decode("utf-8")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import operator
!sudo apt install hunspell-ar
!sudo apt install libhunspell-dev
!sudo apt install aspell
!sudo apt install libaspell-dev
!sudo apt install aspell-ar
!sudo apt install aspell-ar-large
!pip install hunspell
!pip install aspell-python-py3
import hunspell
from google.colab import files
import pandas as pd
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")
!pip install farasapy
from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer

In [ ]:
Books = []
Category= []
BookID = 1
for PageNum in range(200):
  PageNum+=1
  url = 'https://www.arab-books.com//page/'
  url = url+str(PageNum)
  response = requests.get(url)
  Data = BeautifulSoup(response.text, 'html.parser')

  for row in Data.findAll('div',attrs={'class':'excerpt-book'} ):
    Book = {}
    Book['ID'] = BookID
    BookID+=1
    Book['Name']=row.text.replace(" PDF","").replace('\n','')
    Book['link']=row.a['href']+"?view=read"
    url = row.a['href']
    print(url)

    try:

      BookResponse = requests.get(url)
      BookData = BeautifulSoup(BookResponse.text, 'html.parser')
      Mol5sData=BeautifulSoup(BookResponse.text, 'html.parser')
   
    except requests.exceptions.RequestException as e:
      continue
   # Descrip #
    Mol5sData=Mol5sData.findAll('div', attrs={'class': 'entry-content entry clearfix'})[0]
    count=0
    s=""
    for P in Mol5sData:
      if re.findall('<h[0-9]>تحميل.*</h[0-9]>',str(P)):
        continue
      elif re.findall('<h[0-9]>',str(P)):
        count+=1
      elif re.findall('<p>',str(P)) and count!=1:
        s+=str(P)
      if count==3:
        break;
    H=""
    Descc=(re.findall("[ا-ي]+",s))
    for i in range(len(Descc)):
        H += " " + Descc[i]
    Book['Description']=H
    #print(url)
    BookData=BookData.findAll('div', attrs={'class':'book-info'})[0]
    print(BookID)
    Auther = BookData.findAll('li')[0].text.split('\n')
    Book['Auther']=Auther[1].replace('كتب ','')

    Category= []
    for fTest in BookData.findAll('li')[1].text.split('\n'):
      if fTest.find('تحميل كتب'):
        continue
      Category.append(fTest.split('تحميل كتب ')[1])
    Book['Category']=Category
    Book['Language']=BookData.findAll('li')[2].text.split(': ')[1]
    Book['Num Of Pages']=BookData.findAll('li')[3].text.split(': ')[1].replace(' صفحة','')
    Book['Dar Al Nashr']=BookData.findAll('li')[4].text.split(': ')[1]
    Book['Book Size']=BookData.findAll('li')[5].text.split(': ')[1]
    Book['Book Type']=BookData.findAll('li')[6].text.split(': ')[1]
    
    Books.append(Book)

In [212]:
Bookspd= pd.DataFrame(Books)


In [186]:
from google.colab import files
Bookspd.to_csv("booksfinal2.csv")
files.download("booksfinal2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [213]:
Bookspd["Dar Al Nashr"].replace({"-":"غير معرف"}, inplace=True)
Bookspd["Dar Al Nashr"].replace({"":"غير معرف"}, inplace=True)
Bookspd=  Bookspd.drop(['ID'], axis = 1)
Bookspd=  Bookspd.drop(['Language'], axis = 1)
Bookspd=  Bookspd.drop(['Num Of Pages'], axis = 1)
Bookspd=  Bookspd.drop(['Book Size'], axis = 1)
Bookspd=  Bookspd.drop(['Book Type'], axis = 1)


In [214]:
import re
word=[]
listofdrops=[]
for j in range(len(Bookspd["Name"])):
    word=(re.findall("^[ \u0621-\u064A ]+$",Bookspd["Name"][j]))
    if len(word) == 0 :
      Bookspd.drop(j, inplace = True)
      listofdrops.append(j)

In [215]:
import re
Name=[]
matched_word=""
for j in Bookspd['Name']:
 matched_words=re.findall("الا[\u0621-\u064A]+",j)
 if(len(matched_words) > 0 ):
   for j in range(len(matched_words)):
    Name.append(matched_words[j])
Name.remove('الاحوال')

In [216]:
corrected_words1={}
if __name__ == '__main__':
    hobj = hunspell.HunSpell(file_name2,file_name)
    for word in Name:
        if not hobj.spell(word):
          if(len(hobj.suggest(word))>0):
           corrected_words1[word]=hobj.suggest(word)[0]
          else:
             corrected_words1[word]=word
        else:
          corrected_words1[word]=word

    for word in corrected_words1:
      Bookspd['Name']=Bookspd['Name'].str.replace(word,corrected_words1[word])


In [217]:
Cat=[]
Cat_list=[]
Catt=[]
matched_w=""
word=""
for j in range(len(Bookspd['Category'])):
  if j not in listofdrops:
    for i in range(len(Bookspd['Category'][j])):
        word=(Bookspd['Category'][j][i])
        matched_ws=re.findall("ا[\u0621-\u064A^\S]+",word)
        if(len(matched_ws)>0):
          for k in matched_ws: 
            if(len(k) >4):     
              Cat.append(k)
for j in range(len(Cat)):
    Cat_list.append(Cat[j])

In [218]:
Catt=list(set(Cat_list))
corrected_words={}
if __name__ == '__main__':
    hobj = hunspell.HunSpell(file_name2,file_name)
    errors_count = dict()
    for word in Catt:
      if not hobj.spell(word):
         if(len(hobj.suggest(word))>0):
             corrected_words[word]=hobj.suggest(word)[0]
         else:
               corrected_words[word]=word
      else:
          corrected_words[word]=word
    for word in corrected_words:
      for j in range(len(Bookspd['Category'])):
        if j not in listofdrops:
          for i in range(len(Bookspd['Category'][j])):
           Bookspd['Category'][j][i]=Bookspd['Category'][j][i].replace(word,corrected_words[word])

for j in range(len(Bookspd['Category'])):
  if j not in listofdrops:
        for i in range(len(Bookspd['Category'][j])): 
            Bookspd['Category'][j][i]=Bookspd['Category'][j][i].replace("أسلامية","إسلامية")


In [219]:
for j in range(len(Bookspd["Category"])):
  if j not in listofdrops:
    if len(Bookspd["Category"][j]) == 0 or len(Bookspd["Description"][j]) == 0:
        Bookspd.drop(j, inplace = True)

In [220]:
from google.colab import files
Bookspd.to_csv("booksfinal2.csv")
files.download("booksfinal2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
upload = files.upload()

In [223]:
data = pd.read_csv('booksfinal2AfterCleaning.csv', error_bad_lines=False)

In [224]:
data.head()

,Unnamed: 0,Name,link,Description,Auther,Category,Dar Al Nashr
0,4611,رواية السعادة ليس لها تاريخ,https://www.arab-books.com/books/%d8%b1%d9%88%...,احسان محمد عبد القدوس واحد من اهم كتاب والروا...,احسان عبدالقدوس,['روايات عربية'],اخبار اليوم
1,4633,رواية بئر الحرمان,https://www.arab-books.com/books/%d8%b1%d9%88%...,احسان محمد عبد القدوس واحد من اهم كتاب والروا...,احسان عبدالقدوس,['روايات عربية'],مكتبة إحسان عبدالقدوس
2,5840,كتاب صور من حياة التابعين,https://www.arab-books.com/books/%d9%83%d8%aa%...,استمع و قر كتاب الرجل النبيل محمد رسول الله,الكاتب عبد الرحمن رأفت الباشا,['اسلامية'],غير معرف
3,749,كتاب اشكالية فصل الدين عن السياسة,https://www.arab-books.com/books/%d9%83%d8%aa%...,اشكالية فصل الدين عن السياسة يفان سترينسكي تر...,إيفان سترينسكي,['السياسة'],مؤسسة هنداوي
4,5820,كتاب قصة التتار من البداية الى عين جالوت,https://www.arab-books.com/books/%d9%83%d8%aa%...,اعتمد الكاتب البارع راغب السرجاني في كتاب قصة...,الدكتور راغب السرجاني,['التاريخ الإسلامي'],مؤسسة اقرا للنشر والتوزيع والترجمة


In [225]:
data.drop('Unnamed: 0',axis=1,inplace=True)
#data.drop('Unnamed: 0.1',axis=1,inplace=True)
#data.drop('ID',axis=1,inplace=True)

In [226]:
data.head(100)

,Name,link,Description,Auther,Category,Dar Al Nashr
0,رواية السعادة ليس لها تاريخ,https://www.arab-books.com/books/%d8%b1%d9%88%...,احسان محمد عبد القدوس واحد من اهم كتاب والروا...,احسان عبدالقدوس,['روايات عربية'],اخبار اليوم
1,رواية بئر الحرمان,https://www.arab-books.com/books/%d8%b1%d9%88%...,احسان محمد عبد القدوس واحد من اهم كتاب والروا...,احسان عبدالقدوس,['روايات عربية'],مكتبة إحسان عبدالقدوس
2,كتاب صور من حياة التابعين,https://www.arab-books.com/books/%d9%83%d8%aa%...,استمع و قر كتاب الرجل النبيل محمد رسول الله,الكاتب عبد الرحمن رأفت الباشا,['اسلامية'],غير معرف
3,كتاب اشكالية فصل الدين عن السياسة,https://www.arab-books.com/books/%d9%83%d8%aa%...,اشكالية فصل الدين عن السياسة يفان سترينسكي تر...,إيفان سترينسكي,['السياسة'],مؤسسة هنداوي
4,كتاب قصة التتار من البداية الى عين جالوت,https://www.arab-books.com/books/%d9%83%d8%aa%...,اعتمد الكاتب البارع راغب السرجاني في كتاب قصة...,الدكتور راغب السرجاني,['التاريخ الإسلامي'],مؤسسة اقرا للنشر والتوزيع والترجمة
...,...,...,...,...,...,...
95,كتاب الموظف غير المرئي,https://www.arab-books.com/books/%d9%83%d8%aa%...,الموظف غير المر ي استخدام الدافع لاكتشاف مكان...,أدريان جوستيك,['إدارة الأعمال'],مؤسسة هنداوي
96,كتاب الواسطة في احوال مالطة,https://www.arab-books.com/books/%d9%83%d8%aa%...,الواسطة في احوال مالطة احمد فارس الشدياق تقع ...,أحمد الشدياق,['أدب الرحلات'],مؤسسة هنداوي
97,رواية انتقام للكاتب روبرت بار,https://www.arab-books.com/books/%d8%b1%d9%88%...,انتقام روبرت بار ترجمه نبيل العادلي وراجع مصط...,الكاتب روبرت بار,['الروايات العالمية المترجمة'],مؤسسة هنداوي
98,كتاب تاريخ آل رومانوف,https://www.arab-books.com/books/%d9%83%d8%aa%...,انتهت سلالة عا لة رومانوف رومانوف ذات الثلاثم...,سليم قبعين,['التاريخ الحديث'],هندواي


In [ ]:
data.head()

In [227]:
data_text = data[['Description']];
data_text['index'] = data_text.index
documents = data_text
data_text.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Description,index
0,احسان محمد عبد القدوس واحد من اهم كتاب والروا...,0
1,احسان محمد عبد القدوس واحد من اهم كتاب والروا...,1
2,استمع و قر كتاب الرجل النبيل محمد رسول الله,2
3,اشكالية فصل الدين عن السياسة يفان سترينسكي تر...,3
4,اعتمد الكاتب البارع راغب السرجاني في كتاب قصة...,4


In [228]:
stopwords_list = stopwords.words("arabic")

In [ ]:
stemmer = FarasaStemmer()
sample = """قاتل المسلمون الكفار في غزوة احد وانتصر الكفار لعصيان المسلمون لاوامر الرسول صلى الله عليه وسلم"""
stemmed_text = stemmer.stem(sample) 
print(stemmed_text)

In [230]:
def lemmatize_stemming(text):
    return stemmer.stem(text)

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in stopwords_list and len(token) > 2:
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            result.append(lemmatize_stemming(token))
            
    return result

In [ ]:
document_num = 2791
doc_sample = documents[documents['index'] == document_num].values[0][0]

doc_sample
print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

In [ ]:
processed_docs = documents['Description'].map(lambda x: preprocess(x))
processed_docs[:10]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
len(dictionary)
dictionary.filter_extremes(no_below=20, no_above=0.3, keep_n=100000)
len(dictionary)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[document_num]

In [ ]:
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

In [ ]:
from gensim import corpora, models

# TODO
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics = 10, id2word = dictionary, passes = 50)
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics = 10, id2word = dictionary, passes = 50)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} \nWord: {}".format(idx, topic))
    print("\n")

In [ ]:
processed_docs[2791]
document_num = 2791
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

In [ ]:
unseen_document = "العلم والايمان سلاحا المتعلم"

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
threshold = 0.1
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    if score < threshold: break
    print("Score: {}\n Topic: {}".format(score, lda_model.print_topic(index, 5)))
    print()